In [1]:
import torch
import cv2
import os
import numpy as np

# --- Configurations ---
model_path = r'D:\ai\subhi\my_model_teeth_height.pt'     # Your model path
images_dir = r'D:\ai\subhi\undistorted gear side view images'  # Your images folder
output_dir = r'D:\ai\subhi\side view gear bounding boxes'      # Folder to save output images

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load your PyTorch model
model = torch.load(model_path)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def preprocess_image(img):
    # Convert BGR to RGB, resize, normalize, convert to tensor
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (640, 640))  # Adjust input size as needed
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0)  # Add batch dim
    return img_tensor.to(device)

def draw_boxes(img, boxes, scores):
    for box, score in zip(boxes, scores):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{score:.2f}"
        # Draw filled rectangle behind text for readability
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), (0, 255, 0), -1)
        cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 2)
    return img

conf_threshold = 0.5

for img_file in os.listdir(images_dir):
    img_path = os.path.join(images_dir, img_file)
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read {img_file}")
        continue

    input_tensor = preprocess_image(img)

    with torch.no_grad():
        preds = model(input_tensor)[0]  # Change this if your model returns differently

    # Assuming preds has 'boxes' and 'scores' attributes or keys; adjust if needed
    boxes = preds['boxes'].cpu().numpy()
    scores = preds['scores'].cpu().numpy()

    # Filter by confidence threshold
    keep = scores >= conf_threshold
    boxes = boxes[keep]
    scores = scores[keep]

    img_with_boxes = draw_boxes(img, boxes, scores)

    output_path = os.path.join(output_dir, img_file)
    cv2.imwrite(output_path, img_with_boxes)
    print(f"Saved: {output_path}")


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL ultralytics.nn.tasks.DetectionModel was not an allowed global by default. Please use `torch.serialization.add_safe_globals([ultralytics.nn.tasks.DetectionModel])` or the `torch.serialization.safe_globals([ultralytics.nn.tasks.DetectionModel])` context manager to allowlist this global if you trust this class/function.

Check the documentation of to